# Determining optimal banking strategy in the game show "The Weakest Link"
The Weakest Link has a group of people answer questions in a circle. For each correct answer, the team can go up the totem pole of values they can "bank." Once a member banks or gets a question wrong, the amount that can be banked returns to 0. Here is the money ladder: [0, 250, 500, 1000, 1500, 2250, 3000, 4000, 5000]

For example, if Player 1 gets a question correct, Player2 can bank 100 pounds. If Player2 gets the next question correct, Player3 can bank 250 pounds, and so on. But if Player2 gets the question wrong, Player3 cannot bank anything and the team must start climbing the money ladder all over again. What is the optimal stopping strategy? Should players hedge their bets after reaching 500 pounds? 4000? What if they are really talented? Or not very skillful?

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go

%matplotlib inline

## Create game function

In [9]:
# These are the payment levels
pounds = [0, 250, 500, 1000, 1500, 2250, 3000, 4000, 5000]

def runGame(skill, threshold, N):
    '''
    Run a hypothetical round of The Weakest Link. Skill is the likelihood of getting an answer correct,
    threshold is the number of turns a player waits before banking, and N is the number of questions asked.
    '''
    now = 0
    bank = 0
    for i in range(N):
        questionRight = random.randint(1, 101) <= skill
        if questionRight:
            now += 1
        else:
            now = 0
        
        if now == threshold:
            bank += pounds[now]
            now = 0
    
    if now < threshold:
        bank += pounds[now]
    
    return bank

## Testing

In [10]:
# N is the number of questions, B is the number of tests to average over
N, B = 100, 500 

skill = range(101)
threshold = range(0, 9)
results = np.zeros([len(skill), len(threshold)])
for i, s in enumerate(skill):
    for j, t in enumerate(threshold):
        allTests = np.array([runGame(s, t, N=N) for _ in range(B)])
        results[i, j] = allTests.mean()

## Analysis

In [11]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
ax.set_title('Heatmap of winnings when comparing skill to turns waited')
ax.set_xlabel('Number of turns before banking')
ax.set_ylabel('Percent skill')
plotly_fig = tls.mpl_to_plotly(fig)

trace = dict(z=results, type='heatmap', zmin=0)
plotly_fig.add_traces([trace])

plotly_fig['layout']['xaxis'].update({'autorange':True})
plotly_fig['layout']['yaxis'].update({'autorange':True})

py.iplot(plotly_fig, filename='weakestLinkHeatMap')

In [12]:
rec_stop = [np.argmax(results[i, :]) for i in range(len(skill))]
rec_value = [results[i, :].max() for i in range(len(skill))]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
axes[0].barh(width=rec_stop, y=[i for i in range(len(skill))])
axes[0].set_ylabel('percent skill')
axes[0].set_xlabel('recommended number of turns players should wait')
axes[1].set_ylabel('percent skill')
axes[1].set_xlabel('total winnings')
axes[1].barh(width=rec_value, y=[i for i in range(len(skill))])
fig.suptitle('Results of optimal performance')
plotly_fig = tls.mpl_to_plotly(fig)
py.iplot(plotly_fig, filename='weakestLinkOptimal')

In [13]:
data = [go.Contour(z=results, ncontours=25)]
layout = go.Layout(
    title="Contour Map of Winnings", 
    xaxis={'title':'number of turns player waits'}, 
    yaxis={'title':'total winnings'},
    autosize=False,
    width=800,
    height=800,
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='weakestLinkCountour')